In [ ]:

# BSDE mock + dynamic policy (backward recursion) + Gemini reporting
# - จำลอง λ0(t) และ severity (lognormal)
# - คำนวณ “นโยบายแบบไดนามิก” ด้วย BSDE-style backward recursion (risk-sensitive CE)
# - เทียบกับ “นโยบายคงที่” แบบ grid search เบา ๆ
# - ใช้ Gemini สร้างรายงานเชิงกลยุทธ์จากผลลัพธ์ (mock)

import os
import json
from pathlib import Path
import numpy as np
import pandas as pd
from google import genai

# ------------------- CONFIG -------------------
OUT_DIR = Path("bsde_out_bsde_mock")
OUT_DIR.mkdir(parents=True, exist_ok=True)

CFG = {
    "timezone": "Asia/Bangkok",
    "dt_hours": 1,
    "r_annual": 0.03,             # ไม่ใช้ใน mock BSDE นี้ (โฟกัส cost+loss)
    "eta_per_million": 0.5,       # risk aversion บนหน่วยล้านบาท
    "scale": "Million THB",
    "budget_total_thb": 10_000_000.0,

    # horizon (mock 7 วัน hourly)
    "horizon_start_utc": "2021-04-01T00:00:00Z",
    "horizon_end_utc": "2021-04-07T00:00:00Z",

    # mock λ0(t) และ severity
    "lam0_base_per_hour": 0.03,
    "lam0_amp": 0.02,
    "severity_lognormal_mu": float(np.log(1.2)),
    "severity_lognormal_sigma": 0.6,  # บนหน่วย "ล้านบาท"

    # control effectiveness
    "k1": 1.2,  # u1 ลดความถี่: gamma1(u1)=exp(-k1*u1)
    "k2": 1.0,  # u2 ลดความรุนแรง: gamma2(u2)=exp(-k2*u2)

    # cost ต่อชั่วโมง (หน่วยบาท) → แปลงเป็นล้านบาทภายใน
    "c1_b_thb_per_hour": 2.0e5,   # quadratic: 0.5*b*u^2
    "c2_b_thb_per_hour": 1.0e5,

    # grid และ sample
    "u_grid_bsde": 5,             # 0.0..1.0 แบ่ง 5 ค่า (0.0, 0.25, 0.5, 0.75, 1.0)
    "samples_per_step": 300,      # จำลองต่อสเต็ปเพื่อคำนวณ E[exp(eta*(cost+loss))]
    "u_grid_static": 5,           # สำหรับ static policy เทียบกัน
}

MODEL = "gemini-2.5-flash"
API_KEY = ""

# ------------------- HELPERS -------------------
def _to_md_table(df: pd.DataFrame) -> str:
    if df.empty:
        return ""
    cols = [str(c) for c in df.columns]
    lines = []
    lines.append("|" + "|".join(cols) + "|")
    lines.append("|" + "|".join(["---"] * len(cols)) + "|")
    for _, row in df.iterrows():
        lines.append("|" + "|".join("" if pd.isna(v) else str(v) for v in row.tolist()) + "|")
    return "\n".join(lines)

def gamma1(u1: float, k1: float) -> float:
    u1 = float(min(max(u1, 0.0), 1.0))
    return float(np.exp(-k1 * u1))

def gamma2(u2: float, k2: float) -> float:
    u2 = float(min(max(u2, 0.0), 1.0))
    return float(np.exp(-k2 * u2))

def hourly_cost_million(u1: float, u2: float, cfg: dict) -> float:
    c1 = 0.5 * cfg["c1_b_thb_per_hour"] * (u1 ** 2)
    c2 = 0.5 * cfg["c2_b_thb_per_hour"] * (u2 ** 2)
    return (c1 + c2) * 1e-6  # บาท → ล้านบาท

def make_horizon_grid(cfg: dict):
    start = pd.Timestamp(cfg["horizon_start_utc"])
    end = pd.Timestamp(cfg["horizon_end_utc"])
    grid = pd.date_range(start, end, freq=f"{cfg['dt_hours']}h", tz="UTC")
    return grid

def mock_lambda_series(grid: pd.DatetimeIndex, cfg: dict) -> np.ndarray:
    n = max(len(grid) - 1, 1)
    x = np.linspace(0, 4 * np.pi, n)
    lam = cfg["lam0_base_per_hour"] + cfg["lam0_amp"] * np.sin(x)
    lam = np.clip(lam, 0.001, None)
    return lam

def severity_sampler_million(n: int, mu: float, sigma: float) -> np.ndarray:
    return np.exp(np.random.normal(mu, sigma, size=int(n)))  # หน่วยล้านบาท

# ------------------- STATIC POLICY (อ้างอิง) -------------------
def static_objective_logU(lam_per_hour: np.ndarray, mu: float, sigma: float, u1: float, u2: float, cfg: dict) -> float:
    """
    คำนวณ log U0 สำหรับนโยบายคงที่: U0 = Π_k E[exp(eta*(cost_k+loss_k(u)))]
    (CE แบบ risk-sensitive) แบบรวดเร็วด้วย Monte Carlo ต่อสเต็ป
    """
    eta = cfg["eta_per_million"]
    dt_h = cfg["dt_hours"]
    g1 = gamma1(u1, cfg["k1"])
    g2 = gamma2(u2, cfg["k2"])
    cost_h = hourly_cost_million(u1, u2, cfg)

    logU = 0.0
    for k in range(len(lam_per_hour)):
        lam_u = max(0.0, g1 * lam_per_hour[k]) * dt_h
        # draw N claims per sample
        Ns = np.random.poisson(lam_u, size=cfg["samples_per_step"])
        losses = []
        for N in Ns:
            if N <= 0:
                losses.append(0.0)
            else:
                losses.append(float(np.sum(g2 * severity_sampler_million(N, mu, sigma))))
        losses = np.asarray(losses, dtype=float)
        y = eta * (cost_h * dt_h + losses)
        m = float(np.max(y))
        log_eexp = m + np.log(np.mean(np.exp(np.clip(y - m, -1000.0, 0.0))))
        logU += float(np.clip(log_eexp, -745.0, 709.0))
    return float(logU)

def find_static_policy(lam_per_hour: np.ndarray, mu: float, sigma: float, cfg: dict):
    grid = np.linspace(0.0, 1.0, cfg["u_grid_static"])
    best = {"u1": 0.0, "u2": 0.0, "logU0": float("+inf")}
    for u1 in grid:
        for u2 in grid:
            logU = static_objective_logU(lam_per_hour, mu, sigma, float(u1), float(u2), cfg)
            if logU < best["logU0"]:
                best = {"u1": float(u1), "u2": float(u2), "logU0": float(logU)}
    return best

# ------------------- BSDE-STYLE BACKWARD POLICY -------------------
def bsde_backward_policy(lam_per_hour: np.ndarray, mu: float, sigma: float, cfg: dict):
    """
    Backward recursion (risk-sensitive):
      U_T = 1  → logU_T = 0
      เลือก u_k เพื่อลด U_k = E[exp(eta*(cost_k+loss_k(u)))] * U_{k+1}
      ⇒ logU_k = min_u { log E[exp(eta*(cost_k+loss_k(u)))] + logU_{k+1} }
    คืนค่าลิสต์ u1*, u2* ต่อเวลา และ logU0
    """
    eta = cfg["eta_per_million"]
    dt_h = cfg["dt_hours"]
    grid_u = np.linspace(0.0, 1.0, cfg["u_grid_bsde"])

    T = len(lam_per_hour)
    u1_star = np.zeros(T, dtype=float)
    u2_star = np.zeros(T, dtype=float)

    logU_next = 0.0  # logU_T
    # เดินย้อนเวลา
    for k in reversed(range(T)):
        lam_k = lam_per_hour[k]
        best_log = float("+inf")
        best_u1, best_u2 = 0.0, 0.0

        for u1 in grid_u:
            g1 = gamma1(float(u1), cfg["k1"])
            lam_u_dt = max(0.0, g1 * lam_k) * dt_h
            for u2 in grid_u:
                g2 = gamma2(float(u2), cfg["k2"])
                cost_h = hourly_cost_million(float(u1), float(u2), cfg)

                # MC สำหรับ log E[exp(eta*(cost+loss))]
                Ns = np.random.poisson(lam_u_dt, size=cfg["samples_per_step"])
                losses = []
                for N in Ns:
                    if N <= 0:
                        losses.append(0.0)
                    else:
                        losses.append(float(np.sum(g2 * severity_sampler_million(N, mu, sigma))))
                losses = np.asarray(losses, dtype=float)
                y = eta * (cost_h * dt_h + losses)

                m = float(np.max(y))
                log_eexp = m + np.log(np.mean(np.exp(np.clip(y - m, -1000.0, 0.0))))
                log_eexp = float(np.clip(log_eexp, -745.0, 709.0))

                cand = log_eexp + logU_next
                if cand < best_log:
                    best_log = cand
                    best_u1, best_u2 = float(u1), float(u2)

        u1_star[k] = best_u1
        u2_star[k] = best_u2
        logU_next = best_log  # ใช้เป็น logU_{k} สำหรับสเต็ปก่อนหน้า

    return {
        "u1_series": u1_star.tolist(),
        "u2_series": u2_star.tolist(),
        "logU0": float(logU_next)
    }

# ------------------- PIPELINE (MOCK BSDE + REPORT) -------------------
def run_bsde_mock_and_report():
    grid = make_horizon_grid(CFG)
    lam0 = mock_lambda_series(grid, CFG)
    mu = CFG["severity_lognormal_mu"]
    sigma = CFG["severity_lognormal_sigma"]

    # Static (อ้างอิง)
    best_static = find_static_policy(lam0, mu, sigma, CFG)
    static_obj = float(np.exp(np.clip(best_static["logU0"], -745.0, 709.0)))

    # BSDE dynamic policy
    dyn = bsde_backward_policy(lam0, mu, sigma, CFG)
    dyn_obj = float(np.exp(np.clip(dyn["logU0"], -745.0, 709.0)))

    # Compose policy object (mock-friendly แต่มีผลลัพธ์จากตัวแกน)
    policy = {
        "config": {
            "timezone": CFG["timezone"],
            "dt_hours": CFG["dt_hours"],
            "r_annual": CFG["r_annual"],
            "eta_per_million": CFG["eta_per_million"],
            "k1": CFG["k1"], "k2": CFG["k2"],
            "scale": CFG["scale"]
        },
        "time_grid_start": grid[0].isoformat(),
        "time_grid_end": grid[-1].isoformat(),
        "steps": int(len(lam0)),
        "lambda0_avg_per_hour": float(np.mean(lam0)),
        "severity_lognormal": {"mu": mu, "sigma": sigma, "unit": "Million THB"},
        # อ้างอิง baseline = policy คงที่ที่ u1=u2=0
        "baseline_E_exp_eta_cost": float(np.exp(static_objective_logU(lam0, mu, sigma, 0.0, 0.0, CFG))),
        "static_optimal": {"u1": best_static["u1"], "u2": best_static["u2"], "objective": static_obj},
        "bsde_dynamic": {
            "objective": dyn_obj,
            "u1_series": dyn["u1_series"],
            "u2_series": dyn["u2_series"]
        },
        "improvement_ratio_static_vs_dyn": (static_obj / dyn_obj) if dyn_obj > 0 else None
    }

    # Preview λ0 (24 ชั่วโมงแรก)
    preview = pd.DataFrame({
        "t_bin_start_utc": grid[:-1][:24],
        "lambda0_per_hour": lam0[:24]
    })
    md_table = _to_md_table(preview)

    # Budget mock
    interventions = {
        "candidates": [
            {"id": "U12.1", "name": "ทีม GTG 24ชม.", "map": "u1"},
            {"id": "U17.1", "name": "ซ่อมบำรุง Sub station", "map": "u2"},
            {"id": "RESERVE", "name": "งบสำรอง", "map": "reserve"}
        ],
        "target_allocation": {
            "total_budget_thb": CFG["budget_total_thb"],
            "U12.1_thb": 5_000_000.0,
            "U17.1_thb": 3_000_000.0,
            "RESERVE_thb": 2_000_000.0
        }
    }

    # Persist JSONs
    (OUT_DIR / "policy_bsde.json").write_text(json.dumps(policy, ensure_ascii=False, indent=2), encoding="utf-8")
    (OUT_DIR / "interventions.json").write_text(json.dumps(interventions, ensure_ascii=False, indent=2), encoding="utf-8")

    # ------------------- Gemini Reporting -------------------
    prompt = (
        "คุณคือ Data/Quant strategist ภาษาไทย ทำหน้าที่แปลผลลัพธ์จาก BSDE-style dynamic policy ให้เป็นนโยบายที่ปฏิบัติได้จริง "
        "ตอบแบบ dev/data expert ทับศัพท์เทคนิค เช่น BSDE, intensity, severity, utility โดยมีตัวเลขอ้างอิงชัดเจน\n\n"
        "## โมเดล/ฮอไรซอน (Mock)\n"
        f"- Horizon: {policy['time_grid_start']} → {policy['time_grid_end']} | steps={policy['steps']}\n"
        f"- avg λ₀/ชั่วโมง: {policy['lambda0_avg_per_hour']:.4f}\n"
        f"- Severity lognormal(mu={mu:.4f}, sigma={sigma:.2f}, unit={CFG['scale']})\n"
        f"- eta(per {CFG['scale']}): {CFG['eta_per_million']}\n\n"
        "## วัตถุประสงค์ (risk-sensitive, CE of cost)\n"
        f"- baseline (u1=0,u2=0): {policy['baseline_E_exp_eta_cost']:.4f}\n"
        f"- static optimal: u1={policy['static_optimal']['u1']:.2f}, u2={policy['static_optimal']['u2']:.2f}, "
        f"objective={policy['static_optimal']['objective']:.4f}\n"
        f"- BSDE dynamic: objective={policy['bsde_dynamic']['objective']:.4f}, "
        "ได้ลำดับ u1*(t), u2*(t) ตามเวลา\n"
        f"- improvement(static→dynamic): {policy['improvement_ratio_static_vs_dyn']:.4f}\n\n"
        "## λ_t (ย่อ – 24 ชม.แรก)\n"
        f"{md_table}\n\n"
        "## โจทย์เพื่อสรุปกลยุทธ์\n"
        "- เปรียบเทียบ Static vs Dynamic ว่าควรใช้เมื่อไร พร้อม if–then rule บน threshold ของ λ_t\n"
        "- ผูกงบประมาณ 10 ล้านบาท/ปีเข้ากับ u1 (ลดความถี่) และ u2 (ลดความรุนแรง)\n"
        "- สรุป Budget Allocation ตามตัวอย่าง: "
        f"U12.1 {interventions['target_allocation']['U12.1_thb']:.0f} บาท, "
        f"U17.1 {interventions['target_allocation']['U17.1_thb']:.0f} บาท, "
        f"สำรอง {interventions['target_allocation']['RESERVE_thb']:.0f} บาท\n"
        "- ใส่เหตุผลเชิงปริมาณ: แม้บาง risk severity สูง แต่เหตุการณ์ GTG Trip เกิดถี่กว่า → expected loss สูงกว่า → u1 priority\n"
        "- ห้ามส่งโค้ด/ห้าม code fence"
    )

    client = genai.Client(api_key=API_KEY)
    resp = client.models.generate_content(model=MODEL, contents=prompt)
    text = getattr(resp, "text", str(resp)).strip()
    (OUT_DIR / "final_output_bsde.md").write_text(text, encoding="utf-8")

# เรียก pipeline (ไม่ต้องใช้ if __main__)
run_bsde_mock_and_report()